In [1]:
#目的不改变网络的定义代码，也不需要在forward函数中return某个感兴趣层的输出
#register_forward_hook()函数必须在forward（）函数调用之前被使用

# example 
import torch
import torch.nn as nn
class TestForHook(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_1 = nn.Linear(in_features=2, out_features=2)
        self.linear_2 = nn.Linear(in_features=2, out_features=1)
        self.relu = nn.ReLU()
        self.relu6 = nn.ReLU6()      #ReLU6(x)=min(max(0,x),6)   值域0--6
        self.initialize()  #调用initialize函数对所有层进行初始化。

    def forward(self, x):
        linear_1 = self.linear_1(x)
        linear_2 = self.linear_2(linear_1)
        relu = self.relu(linear_2)
        relu_6 = self.relu6(relu)
        layers_in = (x, linear_1, linear_2)
        layers_out = (linear_1, linear_2, relu)
        return relu_6, layers_in, layers_out
    def initialize(self):
        """ 定义特殊的初始化，用于验证是不是获取了权重"""
        self.linear_1.weight = torch.nn.Parameter(torch.FloatTensor([[1, 1], [1, 1]]))
        self.linear_1.bias = torch.nn.Parameter(torch.FloatTensor([1, 1]))
        self.linear_2.weight = torch.nn.Parameter(torch.FloatTensor([[1, 1]]))
        self.linear_2.bias = torch.nn.Parameter(torch.FloatTensor([1]))
        return True
# 定义用于获取网络各层输入输出tensor的容器,并定义module_name用于记录相应的module名字
module_name = []
features_in_hook = []
features_out_hook = []

# hook函数负责将获取的输入输出添加到feature列表中,并提供相应的module名字
# fea_in当前forward 的输入,fea_out当前forward的输出
def hook(module, fea_in, fea_out):  # hook,自己定义钩子函数的名字
    print("hooker working")
    module_name.append(module.__class__)
    features_in_hook.append(fea_in)
    features_out_hook.append(fea_out)
    return None
# children()与modules()都是返回网络模型里的组成元素，
# 但是children()返回的是最外层的元素，modules()返回的是所有的元素，包括不同级别的子元素。
net = TestForHook()
net_chilren = net.children()   #iterator
print(net)

TestForHook(
  (linear_1): Linear(in_features=2, out_features=2, bias=True)
  (linear_2): Linear(in_features=2, out_features=1, bias=True)
  (relu): ReLU()
  (relu6): ReLU6()
)


In [2]:
for child in net_chilren:
    if not isinstance(child, nn.ReLU6):  #除了ReLU6  其余的都注册
        # register_forward_hook, pytorch 提供钩子注册函数
        child.register_forward_hook(hook=hook)

In [3]:
x=torch.rand(2,2)
print(x)
out, features_in_forward, features_out_forward = net(x)

tensor([[0.7218, 0.2633],
        [0.9596, 0.3500]])
hooker working
hooker working
hooker working


In [4]:
out

tensor([[4.9701],
        [5.6192]], grad_fn=<HardtanhBackward0>)

In [5]:
print("*"*5+"test")
features_in_forward

*****test


(tensor([[0.7218, 0.2633],
         [0.9596, 0.3500]]),
 tensor([[1.9851, 1.9851],
         [2.3096, 2.3096]], grad_fn=<AddmmBackward>),
 tensor([[4.9701],
         [5.6192]], grad_fn=<AddmmBackward>))

In [6]:
features_out_forward

(tensor([[1.9851, 1.9851],
         [2.3096, 2.3096]], grad_fn=<AddmmBackward>),
 tensor([[4.9701],
         [5.6192]], grad_fn=<AddmmBackward>),
 tensor([[4.9701],
         [5.6192]], grad_fn=<ReluBackward0>))

In [7]:
print(features_in_hook)
print(features_out_hook)
print(module_name)

[(tensor([[0.7218, 0.2633],
        [0.9596, 0.3500]]),), (tensor([[1.9851, 1.9851],
        [2.3096, 2.3096]], grad_fn=<AddmmBackward>),), (tensor([[4.9701],
        [5.6192]], grad_fn=<AddmmBackward>),)]
[tensor([[1.9851, 1.9851],
        [2.3096, 2.3096]], grad_fn=<AddmmBackward>), tensor([[4.9701],
        [5.6192]], grad_fn=<AddmmBackward>), tensor([[4.9701],
        [5.6192]], grad_fn=<ReluBackward0>)]
[<class 'torch.nn.modules.linear.Linear'>, <class 'torch.nn.modules.linear.Linear'>, <class 'torch.nn.modules.activation.ReLU'>]
